# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 1:59PM,242351,10,8295390,Eye Pharma Inc,Released
1,Jul 20 2022 1:54PM,242350,16,8299670,Sartorius Bioprocess Solutions,Released
2,Jul 20 2022 1:54PM,242350,16,8299673,Sartorius Bioprocess Solutions,Released
3,Jul 20 2022 1:54PM,242350,16,8299711,Sartorius Bioprocess Solutions,Released
4,Jul 20 2022 1:52PM,242349,20,ALUR283857100,Alumier Labs Inc.,Released
5,Jul 20 2022 1:52PM,242349,20,ALUR809482925,Alumier Labs Inc.,Released
6,Jul 20 2022 1:52PM,242349,20,ALU0007353,Alumier Labs Inc.,Released
7,Jul 20 2022 1:52PM,242349,20,ALUR543563265,Alumier Labs Inc.,Released
8,Jul 20 2022 1:46PM,242347,22,621455,"NBTY Global, Inc.",Executing
9,Jul 20 2022 1:41PM,242346,10,CTF0011293-BO,"Citieffe, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,242346,Released,7
37,242347,Executing,1
38,242349,Released,4
39,242350,Released,3
40,242351,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242346,NaN,7.0
242347,1.0,NaN
242349,NaN,4.0
242350,NaN,3.0
242351,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242224,1.0,0.0
242252,0.0,1.0
242263,0.0,12.0
242281,5.0,27.0
242282,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242224,1,0
242252,0,1
242263,0,12
242281,5,27
242282,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242224,1,0
1,242252,0,1
2,242263,0,12
3,242281,5,27
4,242282,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242224,1,
1,242252,,1
2,242263,,12
3,242281,5,27
4,242282,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 1:59PM,242351,10,Eye Pharma Inc
1,Jul 20 2022 1:54PM,242350,16,Sartorius Bioprocess Solutions
4,Jul 20 2022 1:52PM,242349,20,Alumier Labs Inc.
8,Jul 20 2022 1:46PM,242347,22,"NBTY Global, Inc."
9,Jul 20 2022 1:41PM,242346,10,"Citieffe, Inc."
16,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd"
61,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.
69,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC"
70,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc."
71,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 20 2022 1:59PM,242351,10,Eye Pharma Inc,,1
1,Jul 20 2022 1:54PM,242350,16,Sartorius Bioprocess Solutions,,3
2,Jul 20 2022 1:52PM,242349,20,Alumier Labs Inc.,,4
3,Jul 20 2022 1:46PM,242347,22,"NBTY Global, Inc.",1,
4,Jul 20 2022 1:41PM,242346,10,"Citieffe, Inc.",,7
5,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",2,43
6,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,,8
7,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",,1
8,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",,1
9,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:59PM,242351,10,Eye Pharma Inc,1,
1,Jul 20 2022 1:54PM,242350,16,Sartorius Bioprocess Solutions,3,
2,Jul 20 2022 1:52PM,242349,20,Alumier Labs Inc.,4,
3,Jul 20 2022 1:46PM,242347,22,"NBTY Global, Inc.",,1
4,Jul 20 2022 1:41PM,242346,10,"Citieffe, Inc.",7,
5,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",43,2
6,Jul 20 2022 1:28PM,242344,10,Eywa Pharma Inc.,8,
7,Jul 20 2022 12:58PM,242342,10,"Nextsource Biotechnology, LLC",1,
8,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1,
9,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:59PM,242351,10,Eye Pharma Inc,1,
1,Jul 20 2022 1:54PM,242350,16,Sartorius Bioprocess Solutions,3,
2,Jul 20 2022 1:52PM,242349,20,Alumier Labs Inc.,4,
3,Jul 20 2022 1:46PM,242347,22,"NBTY Global, Inc.",,1
4,Jul 20 2022 1:41PM,242346,10,"Citieffe, Inc.",7,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:59PM,242351,10,Eye Pharma Inc,1.0,NaN
1,Jul 20 2022 1:54PM,242350,16,Sartorius Bioprocess Solutions,3.0,NaN
2,Jul 20 2022 1:52PM,242349,20,Alumier Labs Inc.,4.0,NaN
3,Jul 20 2022 1:46PM,242347,22,"NBTY Global, Inc.",NaN,1.0
4,Jul 20 2022 1:41PM,242346,10,"Citieffe, Inc.",7.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 20 2022 1:59PM,242351,10,Eye Pharma Inc,1.0,0.0
1,Jul 20 2022 1:54PM,242350,16,Sartorius Bioprocess Solutions,3.0,0.0
2,Jul 20 2022 1:52PM,242349,20,Alumier Labs Inc.,4.0,0.0
3,Jul 20 2022 1:46PM,242347,22,"NBTY Global, Inc.",0.0,1.0
4,Jul 20 2022 1:41PM,242346,10,"Citieffe, Inc.",7.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3150155,79.0,5.0
12,242336,1.0,0.0
16,1211514,7.0,0.0
18,242224,0.0,1.0
19,2180803,52.0,4.0
20,484683,10.0,3.0
21,969299,4.0,0.0
22,484676,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3150155,79.0,5.0
1,12,242336,1.0,0.0
2,16,1211514,7.0,0.0
3,18,242224,0.0,1.0
4,19,2180803,52.0,4.0
5,20,484683,10.0,3.0
6,21,969299,4.0,0.0
7,22,484676,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,79.0,5.0
1,12,1.0,0.0
2,16,7.0,0.0
3,18,0.0,1.0
4,19,52.0,4.0
5,20,10.0,3.0
6,21,4.0,0.0
7,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,79.0
1,12,Released,1.0
2,16,Released,7.0
3,18,Released,0.0
4,19,Released,52.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Executing,5.0,0.0,0.0,1.0,4.0,3.0,0.0,1.0
Released,79.0,1.0,7.0,0.0,52.0,10.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Executing,5.0,0.0,0.0,1.0,4.0,3.0,0.0,1.0
1,Released,79.0,1.0,7.0,0.0,52.0,10.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Executing,5.0,0.0,0.0,1.0,4.0,3.0,0.0,1.0
1,Released,79.0,1.0,7.0,0.0,52.0,10.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()